# 预测

Apollo预测模块资源：https://github.com/ApolloAuto/apollo/tree/master/modules/prediction

预测有两种方法，基于模型的预测和数据驱动预测

## 模型预测

根据运动学模型，以及持续观察，做出预测；

## 数据驱动预测

适用机器学习算法，通过观察结果来训练模型，就可以在现实世界中利用此模型去做出预测；

![](./predict01.png)

数据驱动方法的优点是，训练数据越多，模型效果越好；

基于模型方法优点，很直观，结合现有的物理知识及交通法规，和其他人类行为方面知识；

这两种方法（基于模型或数据驱动）无好坏之分，但是在某些情况下，其中一种比另一种更有用。

## 车道序列模型

用车道序列描述障碍物行为，Apollo用车道序列号组合描述车辆行为

![](./predict02.png)

为了预测物体的运动，也需要知道物体的状态，人类行驶过程，观察一个物体的朝向，位置、速度、加速度预测下一步运动状态

无人驾驶汽车，除了位置、速度、加速度外，还有车道段内物体的位置，

![](./predict03.png)

还有之前时间间隔的状态信息，以做出更准确预测；

使用车道序列框架的目标，是为道路上的物体生产轨迹，这是一个复杂的问题；

![](./predict04.png)

通常将其分解为一系列小的问题分析，比如车在车道段0，有两个选择，停在0右转或左转到1，将预测问题简化为选择问题；

可以结合训练模型，计算每个车道序列的概率来进行选择；



我们使用观测数据进行经验性训练，在训练中，将真实的车辆行为提供给模型，不仅包括车道段和对象的状态；

还包括对象最终选择哪条车道序列；随着记录随着时间的增加，模型可以自我迭代更新，精确度不断提升；

每个记录都由观察对象跟随的车道段序列，和对象的相关状态，在每个时间点，对象占用一段并具有特定的状态；

整个记录由一系列车道段和对象的相关状态组成

![](./predict05.png)



## 递归神经网路

是一种利用时间序列数据特征的一种预测方法；

可以建立多个MLP，每个MLP单元将序列的一个元素作为输入，提取抽象特征，并预测序列的下一个元素作为输出；

将序列元素前一个MLP的输出，也作为当前元素的输入，进行预测，这就是RNN的结构；

![](./predict06.png)

Apollo使用RNN建立一个模型和来预测车辆的目标车道；

Apollo为车道序列提供一个RNN模型，为相关对象状态提供另一个RNN2模型，链接这两个模型RNN1和RNN2的输出，

并将它们馈送到另一个神经网络，该神经网络会估计每个车道序列的概率；

![](./predict07.png)

我们使用现有的记录，包括相关的对象状态和一个标签，在训练中比较网络输出和真伪标记，并使用反向传播来训练网络；

## 轨迹生成

轨迹生成是预测的最后一步，一旦我们预测到物体的车道序列，就可以预测物体的轨迹，

在任何两点A和B之间，有无数种可能的轨迹，可以先通过设置约束条件，去除大部分候选轨迹，
比如假设汽车和目标车道的中心对齐，去除无法实际运行的轨迹，考虑车辆当前的速度和加速度(乘客舒适感)；

车辆在两点的位置和方位，这两个状态是运动模型的初始状态和最终状态，可以使用这两个条件来拟合一个多项式模型；
大多数情况下，这种多项式可以预测